In [1]:
from poloniex import Poloniex

polo = Poloniex()

In [2]:
markets = polo.returnTicker()
pairs = []
for name, thinks in markets.items():
    pairs.append(name)
print pairs

[u'USDT_REP', u'BTC_CVC', u'BTC_XVC', u'BTC_PINK', u'BTC_SYS', u'BTC_EMC2', u'BTC_RADS', u'BTC_SC', u'BTC_MAID', u'BTC_BCH', u'BTC_GNT', u'BTC_BCN', u'BTC_REP', u'BTC_BCY', u'BTC_GNO', u'XMR_NXT', u'USDT_ZEC', u'BTC_FCT', u'BTC_GAS', u'USDT_ETH', u'USDT_BTC', u'BTC_LBC', u'BTC_DCR', u'USDT_ETC', u'ETH_OMG', u'BTC_AMP', u'BTC_XPM', u'BTC_NXT', u'BTC_VTC', u'ETH_STEEM', u'XMR_BLK', u'BTC_PASC', u'XMR_ZEC', u'BTC_GRC', u'BTC_NXC', u'BTC_BTCD', u'BTC_LTC', u'BTC_DASH', u'BTC_STORJ', u'ETH_ZEC', u'BTC_ZEC', u'BTC_BURST', u'ETH_ZRX', u'BTC_BELA', u'BTC_STEEM', u'BTC_ETC', u'BTC_ETH', u'BTC_HUC', u'BTC_STRAT', u'BTC_LSK', u'BTC_EXP', u'BTC_CLAM', u'ETH_REP', u'XMR_DASH', u'ETH_CVC', u'USDT_BCH', u'BTC_ZRX', u'USDT_DASH', u'BTC_BLK', u'BTC_XRP', u'USDT_NXT', u'BTC_NEOS', u'BTC_OMG', u'BTC_BTS', u'BTC_DOGE', u'ETH_GNT', u'BTC_SBD', u'ETH_GNO', u'BTC_XCP', u'USDT_LTC', u'BTC_BTM', u'USDT_XMR', u'ETH_LSK', u'BTC_OMNI', u'BTC_NAV', u'BTC_FLDC', u'ETH_BCH', u'BTC_XBC', u'BTC_DGB', u'XMR_BTCD', u'BT

In [3]:
# funciones necesarias para el manejo del tiempo
from datetime import datetime, timedelta
from time import sleep
from tusp import string2ts
from bot import prepareData
import numpy as np
import pylab as pl
%matplotlib inline

class PairData:
    
    def __init__(self, pair, period, weeks = 4.3*3, backto = None):
            
        self.pair = pair
        self.period = period
        delta = timedelta(weeks = weeks)
        self.delta = delta
        start = ""
        end = ""

        # Sí el tiempo final es diferente al actual...
        if backto != None:
            to = datetime.strptime(backto, "%Y-%m-%d %H:%M:%S")
            self.to = to
            start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
            
            tf = to + delta
            self.tf = tf
            end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
            
        else:
            tf = datetime.now()
            self.tf = tf
            end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
            
            to = tf-delta
            self.to = to
            start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
                              

        self.start = start
        self.end = end
        df = prepareData(pair=pair, start=start, end=end, period=int(period))
        
        self.df = df
        self.market_return()
    
    def market_return(self):
        """Calcula el retorno del mercado vía log_retornos
        """
        
        serie = self.df["close"]

        # calculando los log retornos
        log_return = np.log(serie).diff().fillna(0)

        # calculando log retorno acumulado
        cum_logr = log_return.cumsum()
        cum_r = np.exp(cum_logr) - 1
        
        self.l_return = log_return
        self.m_return = cum_r
        
    
    def sortino(self, log = False):
        """ Calcula en ratio de Sortino
        
        """
        
        if log:
            rets = self.l_return
         
        else:
            rets = self.m_return

        semi_var = rets[rets < 0] ** 2
        semi_var = semi_var.sum()/len(rets)
        sortino = np.sqrt(semi_var)

        return rets.mean()/sortino

    def sharpe(self, period_ret = 0.2):
        """
        Calculate the annualised Sharpe ratio of a returns stream 
        based on a number of trading periods, N. N defaults to 252,
        which then assumes a stream of daily returns.

        The function assumes that the returns are the excess of 
        those compared to a benchmark.
        """
        N = self.delta.days
        returns = self.l_return
        excess_ret = returns - period_ret/N
        return np.sqrt(N) * excess_ret.mean() / excess_ret.std()
    
dgb = PairData("BTC_DGB", 3600*4)

print dgb.sharpe()

0.0267068599563


In [4]:
# cálculo del ratio de Sortino del último mes
sortinos = []
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]
for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28)
    sortino = asset.sortino(log = True)
    sortinos.append([pair, sortino])
    #print sortino

sortinos.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sortino del penúltimo mes
sortinos2 = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28, "2017-11-14 00:00:00")
    sortino = asset.sortino(log = True)
    sortinos2.append([pair, sortino])
    #print sortino

sortinos2.sort(key = lambda l:l[1], reverse=True)

In [11]:
print sortinos2[0:6]
print sortinos[0:6]

[[u'BTC_EMC2', 0.28035405904741401], [u'BTC_NXT', 0.26237411369725355], [u'BTC_LTC', 0.17117442743364195], [u'BTC_ARDR', 0.11860477781855779], [u'BTC_STR', 0.11682605144268603], [u'BTC_BTM', 0.098805415838810376]]
[[u'BTC_ETH', 0.34513166415658014], [u'BTC_ZRX', 0.31228800413249852], [u'BTC_SC', 0.30111378630533414], [u'BTC_LSK', 0.27272984468029321], [u'BTC_DGB', 0.26019765687048141], [u'BTC_EXP', 0.26009616324237617]]


In [6]:
# cálculo del ratio de Sharpe del último mes
sharpes = []
for pair in pairs:
    asset = PairData(pair, 3600*4, 4.28)
    sharpe = asset.sharpe()
    sharpes.append([pair, sharpe])

sharpes.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sharpe del penúltimo mes
sharpes2 = []

for pair in pairs:
    asset = PairData(pair, 3600*4, 4.28, "2017-11-14 00:00:00")
    sharpe = asset.sharpe()
    sharpes2.append([pair, sharpe])

sharpes2.sort(key = lambda l:l[1], reverse=True)

In [12]:
print sharpes2[:6]
print sharpes[:6]

[[u'BTC_EMC2', 0.43910293222779245], [u'BTC_NXT', 0.11950063662106654], [u'BTC_SBD', -0.13729856100399526], [u'BTC_BTM', -0.2133393262771488], [u'BTC_RIC', -0.30655369136881516], [u'BTC_STR', -0.37693093775951048]]
[[u'BTC_ZRX', 0.38377186959331044], [u'BTC_SC', 0.36497520060184585], [u'BTC_DGB', 0.20974538996147826], [u'BTC_BURST', 0.18334593179941652], [u'BTC_EXP', 0.12011743291926279], [u'BTC_STR', 0.11681258084026287]]


In [8]:
sortinos3 = [x for x in sortinos if x[0].split("_")[0] == "BTC"]
print sortinos3[0:7]

[[u'BTC_ETH', 0.34513166415658014], [u'BTC_ZRX', 0.31228800413249852], [u'BTC_SC', 0.30111378630533414], [u'BTC_LSK', 0.27272984468029321], [u'BTC_DGB', 0.26019765687048141], [u'BTC_EXP', 0.26009616324237617], [u'BTC_STR', 0.25947813911609841]]


In [9]:
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]

In [10]:
from indicators import rsi
rsi_list = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*24, weeks = 4.3*6)
    RSI1, RSI2 = rsi(asset.df["close"])
    rsi_list.append([pair, RSI1[-1]])
    #print sortino

rsi_list.sort(key = lambda l:l[1], reverse=False)

print rsi_list[0:6]

indicators.py:29: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,min_periods=0,adjust=True,com=14).mean()
  roll_up1 = pd.stats.moments.ewma(up, window_length)
indicators.py:30: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,min_periods=0,adjust=True,com=14).mean()
  roll_down1 = pd.stats.moments.ewma(down.abs(), window_length)
indicators.py:42: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=14,center=False).mean()
  roll_up2 = pd.rolling_mean(up, window_length)
indicators.py:43: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=14,center=False).mean()
  roll_down2 = pd.rolling_mean(down.abs(), window_length)


[[u'BTC_NXT', 40.831961929833916], [u'BTC_VTC', 43.258218582987595], [u'BTC_BTCD', 45.055708110755212], [u'BTC_EMC2', 46.42618972039508], [u'BTC_SBD', 46.644154446367047], [u'BTC_BTS', 46.906073361612272]]
